In [1]:
## Import packages

from collections import OrderedDict

import numpy as np

import torch

from torch.utils.data import DataLoader
from torch.distributions import Normal

from eeyore.data import Iris
from eeyore.models import mlp
from eeyore.mcmc import MetropolisHastings, MALA, PowerPosteriorSampler

from timeit import default_timer as timer
from datetime import timedelta

In [2]:
## Load iris data

iris = Iris()
dataloader = DataLoader(iris, batch_size=150)

In [3]:
## Setup MLP model

hparams = mlp.Hyperparameters(dims=[4, 3, 3])
model = mlp.MLP(hparams=hparams)
model.prior = Normal(torch.zeros(model.num_params(), dtype=model.dtype), np.sqrt(3)*torch.ones(model.num_params(), dtype=model.dtype))

In [4]:
## Setup PowerPosteriorSampler

theta0 = model.prior.sample()
per_chain_samplers = [['MALA', {'step': 0.025}], ['MALA', {'step': 0.025}]]

In [5]:
sampler = PowerPosteriorSampler(model, theta0, dataloader, per_chain_samplers, [0.5, 1])